### Naive Relative Encoding with Incremental Parsing

In [1]:
from src.models.const_tree import C_Tree
from src.encs.constituent import *
from src.utils.constants import *
from src.models.linearized_tree import LinearizedTree
from nltk.tree import Tree

gold = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
enc = C_NaiveDynamicEncoding(separator="_", unary_joiner="+", reverse=True)
Tree.fromstring(gold).pretty_print()


t_encoded = enc.encode(C_Tree.from_string(gold))
print(t_encoded)

t_decoded = enc.decode(t_encoded)
t_decoded = t_decoded.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
Tree.fromstring(str(t_decoded)).pretty_print()



/home/poli/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


              S                          
  ____________|________________________   
 |    |   |       VP                   | 
 |    |   |    ___|_________           |  
INTJ  |   NP  |   |         NP         | 
 |    |   |   |   |     ____|____      |  
 RB   ,  PRP VBD  RB  NNP       NNP    . 
 |    |   |   |   |    |         |     |  
 No   ,   it was n't Black     Monday  . 

-BOS-	-BOS-	-BOS-
No	RB	1_S_INTJ
,	,	1_S
it	PRP	1_S_NP
was	VBD	1_S
n't	RB	2_VP
Black	NNP	2_VP
Monday	NNP	3_NP
.	.	-2*_S
-EOS-	-EOS-	-EOS-

              S                          
  ____________|________________________   
 |    |   |       VP                   | 
 |    |   |    ___|_________           |  
INTJ  |   NP  |   |         NP         | 
 |    |   |   |   |     ____|____      |  
 RB   ,  PRP VBD  RB  NNP       NNP    . 
 |    |   |   |   |    |         |     |  
 No   ,   it was n't Black     Monday  . 



### Test of Relative Encoding with Binarization

In [2]:
gold = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
enc = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=True)

print("[*] Gold Tree")
Tree.fromstring(gold).pretty_print()

constituent_tree = C_Tree.from_string(gold)
constituent_tree = constituent_tree.collapse_unary()
binary_tree = C_Tree.to_binary_right(constituent_tree)
binary_tree = binary_tree.remove_preterminals()

print("[*] Binary Tree")
nltk_t = Tree.fromstring(str(binary_tree))
nltk_t.pretty_print()

print("[*] Labels with relative encoding")
enc = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=True)
t_encoded = enc.encode(binary_tree)
print(t_encoded)

t_decoded = enc.decode(t_encoded)
t_decoded = t_decoded.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
Tree.fromstring(str(t_decoded)).pretty_print()


[*] Gold Tree
                                                                                                                   SINV                                                                                                                                 
                        ____________________________________________________________________________________________|_________________________________________________________________________________________________________________________________   
                       S                                                                                                                         |   |   |                                        |                                                   | 
       ________________|______________________________                                                                                           |   |   |                                        |                                           

### Test traversals

In [3]:
from src.models.const_tree import C_Tree
from src.encs.constituent import *
from src.utils.constants import *
from src.models.linearized_tree import LinearizedTree
from nltk.tree import Tree

print("[*] Original Tree")
gold = "(S (INTJ (RB No)) (PUNCT ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (PUNCT .))"

t = C_Tree.from_string(gold)
t = t.collapse_unary()
Tree.fromstring(str(t)).pretty_print()

# print("[*] Traversal pre-order:")
# C_Tree.preorder(t, fn=(lambda x: print(x.label, end=" ")))

print("[*] Traversal in-order:")
C_Tree.inorder(t, fn=(lambda x: print(x.label, end=" ")))
print("\n\n")
right_bin_tree = C_Tree.to_binary_right(t)
Tree.fromstring(str(right_bin_tree)).pretty_print()
restored = C_Tree.restore_from_binary(right_bin_tree)
Tree.fromstring(str(restored)).pretty_print()

print("\n[*] Traversal post-order:")
C_Tree.postorder(t, fn=(lambda x: print(x.label, end=" ")))
print("\n\n")
left_bin_tree = C_Tree.to_binary_left(t)
Tree.fromstring(str(left_bin_tree)).pretty_print()
restored = C_Tree.restore_from_binary(right_bin_tree)
Tree.fromstring(str(restored)).pretty_print()



[*] Original Tree
                      S                            
    __________________|_________________________    
   |      |     |         VP                    |  
   |      |     |      ___|_________            |   
   |      |     |     |   |         NP          |  
   |      |     |     |   |     ____|____       |   
INTJ+RB PUNCT NP+PRP VBD  RB  NNP       NNP   PUNCT
   |      |     |     |   |    |         |      |   
   No     ,     it   was n't Black     Monday   .  

[*] Traversal in-order:
INTJ+RB No PUNCT , NP+PRP it VBD was RB n't VP NNP Black NP NNP Monday S PUNCT . 


          S                                            
    ______|_____                                        
   |            S*                                     
   |       _____|_________                              
   |      |              S**                           
   |      |      _________|________                     
   |      |     |                 S***                 
   |  